In [16]:
from requests import Session
import pandas as pd
import time
import hashlib
import hmac
import base64
import json

# Import Classified

In [17]:
import sys
sys.path.append('..')
import classified

# GLOBAL

In [18]:
api_passphrase = classified.PASSPHRASE
api_key = classified.KEY
api_secret = classified.SECRET

In [34]:
def authentication(method, endpoint, data_json=""):
    now = int(time.time() * 1000)
    str_to_sign = str(now) + method.upper() + endpoint + data_json
    signature = base64.b64encode(
        hmac.new(api_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest())
    passphrase = base64.b64encode(hmac.new(api_secret.encode('utf-8'), api_passphrase.encode('utf-8'), hashlib.sha256).digest())
    headers = {
        "KC-API-SIGN": signature,
        "KC-API-TIMESTAMP": str(now),
        "KC-API-KEY": api_key,
        "KC-API-PASSPHRASE": passphrase,
        "KC-API-KEY-VERSION": '2',
        "Content-Type": "application/json"
    }
    return headers

# Private API

In [44]:
class PrivateData:
    # https://docs.kucoin.com/ - API Documentaion

    def __init__(self):
        self.apiurl = "https://api.kucoin.com"  # base url for kucoin
        self.session = Session()

    # returns account balance
    def getBalance(self):
        url = self.apiurl + '/api/v1/accounts'
        headers = authentication('get', '/api/v1/accounts')
        r = self.session.get(url, headers=headers)
        data = r.json()["data"]
        df = pd.DataFrame.from_dict(data)
        return df
    
    # returns the full order book for a trading pair
    def getFullOrderBook(self, symbol):  # FIXME
        url = self.apiurl + '/api/v3/market/orderbook/level2'
        headers = authentication(method='get', endpoint='/api/v3/market/orderbook/level2')
        parameters = {"symbol": symbol}
        r = self.session.get(url, headers=headers, params=parameters)
        data = r.json()
        return data
    
    # places a new order
    def placeOrder(self, side, symbol, price, size):
        url = self.apiurl + '/api/v1/orders'
        data = {"clientOid": "01", "side": side, "symbol": symbol, "price": price,"size": size}
        data_json = json.dumps(data)
        headers = authentication('post', '/api/v1/orders', data_json=data_json)
        response = self.session.post(url, headers=headers, data=data_json)
        return response.json()


In [45]:
kucoinAPI = PrivateData()

In [46]:
kucoinAPI.getBalance()

,id,currency,type,balance,available,holds
0,610e0377e962a1000673e64b,USDT,trade,257.29528062,249.28728062,8.008
1,6243a55ad7856d00012e4ded,CGG,trade,34.8506,34.8506,0
2,62439c47f387e7000121c052,OCEAN,trade,21.4401,21.4401,0
3,624396dcd8d7fe000185a864,XTAG,trade,40,40,0
4,62439c2ae8e40d00013e99f6,VRA,trade,640.9041,640.9041,0
5,620eae22e341cd0001f3fbf6,XLM,trade,0.000043,0.000043,0
6,610e0477e962a10006754f94,ETH,trade,0.00000007,0.00000007,0
7,617ccb73c429720001f948f1,ADA,main,0,0,0
8,617ccb9c21d14c0001a1a233,ADA,trade,0,0,0
9,6244482fd126950001728bf2,ALBT,trade,0,0,0


In [49]:
kucoinAPI.placeOrder("buy", "ETH-USDT", "800", "0.01")

{'code': '400100', 'msg': 'validation.createOrder.clientOidDuplicated'}